In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['figure.figsize'] = 9, 6
%load_ext autoreload
%autoreload 2

In [ ]:
from keras import backend as K
K.set_learning_phase(0)
%run survey_lcs.py 64 2 0.25 --no_train --model_type conv --lr 5e-4 --sim_type survey_lcs/n200_periodic --nb_epoch 1 --n_min 200 --n_max 200 --m_max 32 --embedding 64 --filter_length 7 --batch_norm

# Visualizing predictions

In [ ]:
%%time
pred = model.predict({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)}, batch_size=500)

In [ ]:
i = -1

In [ ]:
#inds = ~np.isnan(X_raw[i, :, 0])
inds = np.arange(args.n_min)
i += 1
#i = np.random.choice(inds)
t = np.cumsum(X_raw[i, inds, 0])
m = X[i, inds, 1]
#m = np.random.normal(size=len(t))
e = X_raw[i, inds, 2]

plt.errorbar(t, m, e, None, 'o');
plt.plot(t, pred[i], 'o');

#from gatspy.periodic import LombScargleFast
#gat_resids = np.zeros(len(X))
#model_gat = LombScargleFast(fit_period=True, optimizer_kwds={'period_range': (0.05 * (t.max() - t.min()), 0.95 * (t.max() - t.min())), 'quiet': True}, silence_warnings=True)
#model_gat.fit(t, m)
#print(model_gat.score(model_gat.best_period).item())

#w_r, A_r, phi_r, b_r = pred_gru[i]
#x_r = A_r * np.sin(2 * np.pi * w_r * t + phi_r) + b_r
#plt.plot(t, x_r, '--')

plt.legend(['Original', 'Reconstructed'])
plt.title("MSE: {}, Var: {}".format((np.mean((m - np.squeeze(pred[i])) ** 2)), np.var(m)));

# Visualizing training progress

In [ ]:
import glob
import os

log_files = glob.glob(os.path.join('keras_logs', 'autoencoder/uneven/noise0') + '/*.csv')
logs = [pd.read_csv(f, usecols=['Wall time', 'Step', 'Value']) for f in log_files]
for log, f in zip(logs, log_files):
    label = f[(f.rfind('run_') + 4):f.rfind(',')] + ' ' + ('Validation' if 'val' in f else 'Training')
    log.columns = [label if c == 'Value' else c for c in log.columns]
    log['Wall time'] -= log['Wall time'].min()
time_logs = pd.concat([l.set_index('Wall time').drop('Step', axis=1) for l in logs], axis=1)
step_logs = pd.concat([l.set_index('Step').drop('Wall time', axis=1) for l in logs], axis=1)
step_logs.head(10)

In [ ]:
step_logs.plot?

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

colors = sns.color_palette(n_colors=int(len(step_logs.columns) / 2))
for i, c in enumerate(step_logs.columns):
    step_logs[c].dropna().plot(ax=ax[0], color=colors[int(i / 2)], legend=True, linestyle='-' if 'Training' in c else '--')
    time_logs[c].dropna().plot(ax=ax[1], color=colors[int(i / 2)], legend=True, linestyle='-' if 'Training' in c else '--')
ax[0].set_ylabel('Loss');
ax[1].set_ylabel('Loss');
ax[1].set_xlabel('Wall time (s)');

# Visualizing embeddings

In [ ]:
#%%time
inds = np.arange(args.N_train, args.N_train + args.N_test)
encode_layer = [l for l in model.layers if l.name == 'encoding'][0]
if args.even:
    encode = K.function([model.layers[0].input], [encode_layer.output])
    encoding, = encode([X[inds]])
else:
    inputs = [l for l in model.layers if 'Input' in str(l)]
    encode = K.function([inputs[0].input, inputs[1].input], [encode_layer.output])
    encoding, = encode([X[inds], X[inds, :, 0:1]])

In [ ]:
np.mean((m - pred_i) ** 2)

In [ ]:
S = np.corrcoef(encoding, Y[:encoding.shape[0]], rowvar=0)[:8, 8:]
print(pd.DataFrame(S))
plt.imshow(S, cmap='viridis', interpolation='none')
plt.colorbar()

In [ ]:
sns.jointplot(encoding[:, 5], Y[:encoding.shape[0], 0], kind='hex')

In [ ]:
sns.jointplot(encoding[:, 1], Y[:encoding.shape[0], 1], kind='hex')

In [ ]:
sns.jointplot(encoding[:, 5], Y[:encoding.shape[0], 2], kind='hex')

In [ ]:
sns.jointplot(encoding[:, 3], Y[:encoding.shape[0], 3], kind='hex')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

model = RandomForestRegressor(n_estimators=128)
#model = ElasticNet()
#model = SVR(kernel='rbf')

model.fit(encoding, Y[:encoding.shape[0], 0])

In [ ]:
encoding_test = encode([X[test]])[0]
Y_pred = model.predict(encoding_test)

In [ ]:
#sns.jointplot(Y[test, 0], Y_pred[:, 0])
sns.jointplot(Y[test, 0], Y_pred)

# New data

In [ ]:
import cesium.datasets
from keras.utils.np_utils import to_categorical

data = cesium.datasets.fetch_asas_training()
X_list = [np.c_[t, m, e] for t, m, e in zip(data['times'], data['measurements'], data['errors'])]
sub_array_list = [np.array_split(x, np.arange(args.n_max, len(x), step=args.n_max)) for x in X_list]
sub_array_list = [[el for el in x if len(el) >= args.n_min] for x in sub_array_list]
X_list = [el for x in sub_array_list for el in x]

classnames, indices = np.unique(data['classes'], return_inverse=True)
y = np.repeat(indices, [len(x) for x in sub_array_list])
Y = to_categorical(y, len(classnames))

X_raw = pad_sequences(X_list, value=np.nan, dtype='float', padding='post')
X, scale_params = preprocess(X_raw, args.m_max)

# Remove errors
X = X[:, :, :2]

In [ ]:
encode_layer = [l for l in model.layers if l.name == 'encoding'][0]
inputs = [l for l in model.layers if 'Input' in str(l)]
encode = K.function([inputs[0].input, inputs[1].input], [encode_layer.output])
encoding, = encode([X, X[:, :, 0:1]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

enc_train, enc_test, y_train, y_test = train_test_split(encoding, y, test_size=0.25, random_state=42)

rfc = RandomForestClassifier(n_estimators=512)
rfc.fit(enc_train, y_train)

In [ ]:
print("Training score: {}%".format(100 * rfc.score(enc_train, y_train)))
print("Test score: {}%".format(100 * rfc.score(enc_test, y_test)))